<a href="https://colab.research.google.com/github/srikarreddy1729/DL4ES/blob/main/sandbox/multi_image_training_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. create functions to preprocess the input image:
2. the function should have normalization, resize, creating patches
3. all the images should be converted into lists of patches
4. these lists are sampled and useful lists are created
5. then combine all the sampled into to bigger list,
4. then a numpy array


In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install rasterio
import numpy as np
import rasterio
import keras
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.7 MB/s eta 0:00:00


In [3]:
import os, sys

In [4]:
!pip install gdal


In [5]:
from osgeo import gdal, osr

In [ ]:
#INPUTS - geotif and geojason reference
ps = 'area2_0530_2022_8bands.tif'
roishape = "area2_square.geojson"

rasterfile = gdal.Open(datapath + ps)
print('\nPerforming the clip operation...\n')


warp_options = gdal.WarpOptions(cutlineDSName = roipath + roishape, cropToCutline = True)
rasterfile_new = ps.split('.tif')[0] + '_roi.tif'
ds = gdal.Warp(datapath + rasterfile_new, datapath + ps,  options = warp_options)

cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
projInfo = ds.GetProjection()
spatialRef = osr.SpatialReference()
spatialRef.ImportFromWkt(projInfo)
spatialRefProj = spatialRef.ExportToProj4()
ds = None

print('\nClipped raster input: ', rasterfile_new)
print('Checking spatial reference info\n')
print ("WKT format: " + str(spatialRef))
print ("Proj4 format: " + str(spatialRefProj))
print ("Number of columns: " + str(cols))
print ("Number of rows: " + str(rows))
print ("Number of bands: " + str(bands))


In [41]:
from zmq import backend
#preprocessing
def preprocessing(filelocation):
    # Load the GeoTIFF file
    with rasterio.open(filelocation) as src:
        # Read the TIFF data
        tiff_data = src.read()
        print("total number of nan in original",np.count_nonzero(np.isnan(tiff_data)))

        # Get the shape of the TIFF data
        num_bands, height, width = tiff_data.shape

        print("Original image dimensions:", num_bands, height, width)
        unique_elements, counts_elements = np.unique(tiff_data, return_counts=True)
        print(unique_elements, counts_elements )
        print("total unique",len(counts_elements))

        print(np.min(tiff_data), np.max(tiff_data))


        normalized_image = np.zeros_like(tiff_data, dtype='float32')

        for band, count in enumerate(range(tiff_data.shape[0])):
            band_data = tiff_data[band, :, :]
            band_min = np.min(band_data)
            band_max = np.max(band_data)
            print("band-", count+1,"maximum-",band_max,"minimum-",band_min)
            #print(band_data)
            normalized_band = (band_data - band_min) / (band_max - band_min + 1e-10)
            normalized_image[band, :, :] = normalized_band

        # Calculate the new width and height that are multiples of the patch size
        patch_size = 256  # Replace with your desired patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        input_image = np.moveaxis(normalized_image, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_array = input_image[:new_height, :new_width, :]

    print("total number of nan",np.count_nonzero(np.isnan(cropped_array)))
    print("Cropped array shape:", cropped_array.shape)
    print(np.min(cropped_array), np.max(cropped_array))

    patches = []
    for i in range(0, cropped_array.shape[0], patch_size):
        for j in range(0, cropped_array.shape[1], patch_size):
            patch = cropped_array[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
    print("patches are created")
    return patches

In [7]:
def sampling(training_images, mask_array):
    useful_images = []
    useful_masks = []
    useless = 0
    indexes = []
    for img in range(len(training_images)):
        img_name=training_images[img]
        mask_name = mask_array[img]

        val, counts = np.unique(mask_name, return_counts=True)

        if (1 - (counts[0]/counts.sum())) > 0.05:
          useful_images.append(img_name)
          useful_masks.append(mask_name)
          indexes.append(img)
          print("I am useful")

        else:
          #print("I am useless")
          useless +=1


    print("Total useful images are: ", len(training_images)-useless)
    print(indexes)
    print("Total useless images are: ", useless)

    return useful_images

In [8]:
def combine_lists(lists_of_arrays):
    """
    Combine lists of arrays into a single list.

    Args:
    lists_of_arrays (list): A list of lists, where each inner list contains arrays.

    Returns:
    combined_list (list): A single list containing all arrays from the input lists.
    """
    combined_list = []

    for arr_list in lists_of_arrays:
        combined_list.extend(arr_list)

    return combined_list

# # Example usage:
# list1 = [np.array([1, 2, 3]), np.array([4, 5, 6])]
# list2 = [np.array([7, 8, 9]), np.array([10, 11, 12])]

# lists_of_arrays = [list1, list2]
# combined_list = combine_lists(lists_of_arrays)

# print(combined_list)



In [9]:
#create mask_array

def preprocess_mask(filelocation):


    with rasterio.open(filelocation) as src:
        # Read the TIFF data
        output_mask = src.read()
        num_bands, height, width = src.shape

        # Calculate the new width and height that are multiples of the patch size
        patch_size = 256  # Replace with your desired patch size
        new_width = int(np.floor(width / patch_size)) * patch_size
        new_height = int(np.floor(height / patch_size)) * patch_size

        print("cropped dimensions:", new_height, new_width)

        output_mask = np.moveaxis(output_mask, 0, -1)

        # Crop the input_image to the new dimensions
        cropped_mask = output_mask[:new_height, :new_width, :]

        print("Cropped array shape:", cropped_mask.shape)

        new_mask = np.squeeze(cropped_mask)

    masks = []
    for i in range(0, new_mask.shape[0], patch_size):
        for j in range(0, new_mask.shape[1], patch_size):
            patch = new_mask[i:i+patch_size, j:j+patch_size]
            masks.append(patch)
    return masks


In [ ]:
#combine masks
#apply one hot encoding
from tensorflow.keras.utils import to_categorical
def onehotencoding(labels, num_classes=23):
    return to_categorical(labels, num_classes)
mask_array = np.array(masks)

1. load each image, preprocess and create patches
2. load the mask and create mask-array
3. sample each image-patch with mask
4. get the useful patches
5. combine useful patches and convert into one numpy array
6. combine masks and convert them into one numpy array


In [42]:
patch_1 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0123_2023_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4085 4686
[1.0000e+00 2.0000e+00 3.0000e+00 ... 1.0204e+04 1.0383e+04 1.0888e+04] [1192668  570011  635396 ...       1       1       1]
total unique 6554
1.0 10888.0
band- 1 maximum- 6688.0 minimum- 1.0
band- 2 maximum- 6266.0 minimum- 73.0
band- 3 maximum- 6819.0 minimum- 1.0
band- 4 maximum- 6696.0 minimum- 9.0
band- 5 maximum- 8876.0 minimum- 1.0
band- 6 maximum- 10383.0 minimum- 11.0
band- 7 maximum- 10204.0 minimum- 1.0
band- 8 maximum- 10888.0 minimum- 1.0
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created


In [40]:
patch_2 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0516_2023_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4093 4691
[0.0000000e+00 1.0000000e+00 1.3210372e+00 ... 8.3150000e+03 8.3500000e+03
 9.0132861e+03] [5046768    1581       1 ...       1       1       1]
total unique 37131253
0.0 9013.286
band- 1 maximum- 7270.3057 minimum- 0.0
band- 2 maximum- 6309.0 minimum- 0.0
band- 3 maximum- 5006.967 minimum- 0.0
band- 4 maximum- 6638.0 minimum- 0.0
band- 5 maximum- 6024.095 minimum- 0.0
band- 6 maximum- 7737.994 minimum- 0.0
band- 7 maximum- 7287.0 minimum- 0.0
band- 8 maximum- 9013.286 minimum- 0.0
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created


In [43]:
patch_3 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0516_2023_8bands_composite.tif')

total number of nan in original 0
Original image dimensions: 8 4013 4591
[   0    1    2 ... 9266 9502 9977] [2287920   69191    4276 ...       1       1       1]
total unique 6881
0 9977
band- 1 maximum- 7811 minimum- 0
band- 2 maximum- 7080 minimum- 0
band- 3 maximum- 6078 minimum- 0
band- 4 maximum- 7601 minimum- 0
band- 5 maximum- 6543 minimum- 0
band- 6 maximum- 9977 minimum- 0
band- 7 maximum- 9266 minimum- 0
band- 8 maximum- 9187 minimum- 0
cropped dimensions: 3840 4352
total number of nan 0
Cropped array shape: (3840, 4352, 8)
0.0 1.0
patches are created


In [44]:
patch_4 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0530_2022_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4019 4618
[1.0000000e+00 1.0153766e+00 1.0335821e+00 ... 8.3135000e+03 9.7730000e+03
 1.0610000e+04] [7116    1    1 ...    1    1    1]
total unique 23091645
1.0 10610.0
band- 1 maximum- 3905.5 minimum- 1.0
band- 2 maximum- 5630.5 minimum- 14.0
band- 3 maximum- 6175.5 minimum- 11.0
band- 4 maximum- 6969.5 minimum- 1.0
band- 5 maximum- 7382.0 minimum- 27.0
band- 6 maximum- 10610.0 minimum- 1.0
band- 7 maximum- 8313.5 minimum- 1.0
band- 8 maximum- 7437.0 minimum- 42.0
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created


In [45]:
patch_5 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0617_2023_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4093 4691
[0.00000000e+00 1.00000000e+00 1.00744951e+00 ... 1.47508545e+04
 1.48675693e+04 1.49122227e+04] [5139424     339       1 ...       1       1       1]
total unique 34703699
0.0 14912.223
band- 1 maximum- 10229.461 minimum- 0.0
band- 2 maximum- 8654.425 minimum- 0.0
band- 3 maximum- 10251.252 minimum- 0.0
band- 4 maximum- 10945.41 minimum- 0.0
band- 5 maximum- 14912.223 minimum- 0.0
band- 6 maximum- 11515.0205 minimum- 0.0
band- 7 maximum- 13945.831 minimum- 0.0
band- 8 maximum- 14750.8545 minimum- 0.0
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created


In [46]:
patch_6 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0619_2023_8bands_composite.tif')

total number of nan in original 0
Original image dimensions: 8 4013 4591
[    0     1     2 ... 10374 10394 10750] [1412136 1618170  192589 ...       1       1       1]
total unique 6591
0 10750
band- 1 maximum- 7867 minimum- 0
band- 2 maximum- 7895 minimum- 0
band- 3 maximum- 8909 minimum- 0
band- 4 maximum- 9605 minimum- 0
band- 5 maximum- 10394 minimum- 0
band- 6 maximum- 10750 minimum- 0
band- 7 maximum- 10374 minimum- 0
band- 8 maximum- 10048 minimum- 0
cropped dimensions: 3840 4352
total number of nan 0
Cropped array shape: (3840, 4352, 8)
0.0 1.0
patches are created


In [48]:
patch_7 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0803_2022_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4085 4686
[1.0000000e+00 1.0001994e+00 1.0002265e+00 ... 2.0560777e+04 2.2831994e+04
 3.1133723e+04] [1722168       1       1 ...       1       1       1]
total unique 39859147
1.0 31133.723
band- 1 maximum- 11538.01 minimum- 1.0
band- 2 maximum- 6580.7026 minimum- 43.0
band- 3 maximum- 7336.129 minimum- 1.0
band- 4 maximum- 7969.3013 minimum- 51.0
band- 5 maximum- 12767.986 minimum- 1.0
band- 6 maximum- 11290.568 minimum- 36.0
band- 7 maximum- 31133.723 minimum- 2.0
band- 8 maximum- 16012.023 minimum- 1.0
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created


In [49]:
patch_8 = preprocessing('/content/gdrive/MyDrive/exp/other images/public-archivedwl-5/area2_0913_2022_8bands.tif')

total number of nan in original 0
Original image dimensions: 8 4085 4686
[  330.81818   353.87973   354.5077  ... 53613.      58280.
 65391.     ] [1 1 1 ... 1 1 1]
total unique 21192666
330.81818 65391.0
band- 1 maximum- 65391.0 minimum- 3988.9937
band- 2 maximum- 24658.0 minimum- 3121.3325
band- 3 maximum- 28361.0 minimum- 2164.0
band- 4 maximum- 30501.0 minimum- 1813.0
band- 5 maximum- 46962.0 minimum- 1116.0
band- 6 maximum- 36442.0 minimum- 851.0
band- 7 maximum- 50179.0 minimum- 701.71814
band- 8 maximum- 36001.0 minimum- 330.81818
cropped dimensions: 3840 4608
total number of nan 0
Cropped array shape: (3840, 4608, 8)
0.0 1.0
patches are created
